In [23]:
import numpy as np
import seaborn as sns

from src.tools.cache import NBCache
from src.tools.object import get_custom_attributes

sns.set_style("whitegrid")

In [24]:
SMOTE_LEVEL = 500
cache = NBCache("tvla/t_test", version=f"smote_{SMOTE_LEVEL}")

In [25]:
PV_TVLA = .95
PV_CHI2 = .99

class Tvla:
    default, masked, random = [None] * 3

class TTest(Tvla):
    default = cache.np(f"tvla_default_{int(PV_TVLA*100)}")
    masked = cache.np(f"tvla_masked_{int(PV_TVLA*100)}")
    random = cache.np(f"tvla_random_{int(PV_TVLA*100)}")

class Chi2(Tvla):
    default = cache.np(f"chi2_default_{int(PV_CHI2*100)}")
    masked = cache.np(f"chi2_masked_{int(PV_CHI2*100)}")
    random = cache.np(f"chi2_random_{int(PV_CHI2*100)}")

class Heat(Tvla):
    default = cache.np("heat_default")
    masked = cache.np("heat_masked")
    random = cache.np("heat_random")

In [26]:
def analyse_tvla(tres: Tvla, heat: Tvla):
    for a in get_custom_attributes(Tvla):
        sums = np.zeros(5)
        total = 0

        result = getattr(tres, a)
        heatmap = getattr(heat, a)

        for k1, key_1 in enumerate(result):
            for k2, key_2 in enumerate(key_1):
                h = heatmap[k1][k2][0]
                l, r = key_2

                leak = l * (not r)
                leak_heat = h * leak

                sums += np.array([*key_2, leak, leak_heat, h])
                total += 1

        total = len(result) ** 2

        title = f" TVLA for {a} keys "
        print(f"\n{'=' * len(title)}")
        print(title)
        print(f"{'=' * len(title)}\n")

        print(f"For a total of {total} key combinations: \n")

        print(f"A vs A fails  {int(sums[0])} times.")
        print(f"A vs B fails {int(sums[1])} times. \n")

        print(f"Leakages detected:  {int(sums[2])}")

        print(f"Average mean diff: {sums[4] / total:.4f}")
        print(f"Average mean diff for leakage:  {sums[3] / sums[0]:.4f}")

## $t$-test analysis on ASCAD

In [28]:
analyse_tvla(TTest, Heat)


 TVLA for default keys 

For a total of 65536 key combinations: 

A vs A fails  768 times.
A vs B fails 1170 times. 

Leakages detected:  754
Average mean diff: 0.0406
Average mean diff for leakage:  0.0517

 TVLA for masked keys 

For a total of 65536 key combinations: 

A vs A fails  1024 times.
A vs B fails 1217 times. 

Leakages detected:  1002
Average mean diff: 0.0406
Average mean diff for leakage:  0.0224

 TVLA for random keys 

For a total of 65536 key combinations: 

A vs A fails  768 times.
A vs B fails 1167 times. 

Leakages detected:  748
Average mean diff: 0.0406
Average mean diff for leakage:  0.0248


## $\chi^2$-test analysis on ASCAD

In [27]:
analyse_tvla(Chi2, Heat)


 TVLA for default keys 

For a total of 65536 key combinations: 

A vs A fails  43008 times.
A vs B fails 41859 times. 

Leakages detected:  14786
Average mean diff: 0.0406
Average mean diff for leakage:  0.0133

 TVLA for masked keys 

For a total of 65536 key combinations: 

A vs A fails  39680 times.
A vs B fails 41313 times. 

Leakages detected:  13719
Average mean diff: 0.0406
Average mean diff for leakage:  0.0143

 TVLA for random keys 

For a total of 65536 key combinations: 

A vs A fails  40192 times.
A vs B fails 40983 times. 

Leakages detected:  13858
Average mean diff: 0.0406
Average mean diff for leakage:  0.0136
